In [1]:
import tensorflow.keras
import numpy as np
from tensorflow.keras import layers, models, optimizers

# [4월 28일]
---

## # 글자 수준의 LSTM 텍스트 생성 모델 구현
---
이런 아이디어를 케라스로 구현해 보죠. 먼저 언어 모델을 학습하기 위해 많은 텍스트 데이터가 필요합니다. 위키피디아나 반지의 제왕처럼 아주 큰 텍스트 파일이나 텍스트 파일의 묶음을 사용할 수 있습니다. 이 예에서는 19세기 후반 독일의 철학자 니체의 글을 사용하겠습니다(영어로 번역된 글입니다). 학습할 언어 모델은 일반적인 영어 모델이 아니라 니체의 문체와 특정 주제를 따르는 모델일 것입니다.


### # 데이터 전처리
---

#### # 데이터 파일 파싱
---

In [20]:
import tensorflow.keras
import numpy as np

path = tensorflow.keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()

print('말뭉치 크기:', len(text))

말뭉치 크기: 600893


In [3]:
text[:500]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman? certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--if,\nindeed, it s'

#### # 글자 시퀀스 벡터화
---

In [21]:
maxlen = 60 # 60개 글자로 된 시퀀스 추출
step = 3 # 세 글자씩 건너뛰면서 새로운 시퀀스를 샘플링

sentences = [] # 추출한 시퀀스를 담을 리스트
next_chars = [] # 타깃(시퀀스 다음 글자)을 담을 리스트

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))

chars = sorted(list(set(text))) # 말뭉치에서 고유한 글자를 담은 리스트
print('고유한 글자:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars) # chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리

시퀀스 개수: 200278
고유한 글자: 57


In [24]:
# 원-핫 인코딩

x = np.zeros((len(sentences), maxlen, len(chars)), dtype = bool)
y = np.zeros((len(sentences), len(chars)), dtype = bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

### # 모델 구성
---

In [25]:
from tensorflow.keras import layers, models, optimizers

model = models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = 'softmax'))

model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.01),
              loss = 'categorical_crossentropy')

In [26]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 128)               95232     
                                                                 
 dense_3 (Dense)             (None, 57)                7353      
                                                                 
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


### # 언어 모델 훈련과 샘플링
---
- 훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.
 1. 지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
 2. 특정 온도로 이 확률 분포의 가중치를 조정합니다.
 3.	가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
 4.	새로운 글자를 생성된 텍스트의 끝에 추가합니다.

- 다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다(샘플링 함수입니다)
$$$$

- 지수함수, 로그함수  
 - 밑이 자연상수 e인 지수함수(e^x)의 그래프<br>
https://wooono.tistory.com/214

 - Python Numpy.log()-로그 <br>
https://www.delftstack.com/ko/api/numpy/python-numpy-log/



#### # 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수
---
- 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출

In [27]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#### # 텍스트 생성 루프
---
여기서 볼 수 있듯이 낮은 온도는 아주 반복적이고 예상되는 텍스트를 만듭니다. 하지만 국부적인 구조는 매우 실제와 같습니다. 특히 모든 단어(단어는 글자의 지역 패턴으로 이루어집니다)가 실제 영어 단어입니다. 높은 온도에서 생성된 텍스트는 아주 흥미롭고 놀라우며 창의적이기도 합니다. 이따금 꽤 그럴싸하게 보이는 완전히 새로운 단어를 창조합니다(‘begarmed’와 ‘isharent’ 같은 단어입니다). 높은 온도에서는 국부적인 구조가 무너지기 시작합니다. 대부분의 단어가 어느정도 무작위한 문자열로 보입니다. 확실히 이 네트워크에서는 텍스트 생성에 가장 좋은 온도는 0.5입니다. 항상 다양한 샘플링 전략으로 실험해 봐야합니다! 학습된 구조와 무작위성 사이에 균형을 잘 맞추면 흥미로운 것을 만들 수 있습니다.

더 많은 데이터에서 크고 깊은 모델을 훈련하면 이것보다 훨씬 논리적이고 실제와 같은 텍스트 샘플을 생성할 수 있습니다. 당연히 우연이 아닌 의미 있는 텍스트가 생성된다고 기대하지 마세요. 글자를 연속해서 나열하기 위한 통계 모델에서 데이터를 샘플링한 것뿐입니다. 언어는 의사소통의 수단입니다. 의사소통이 의미하는 것과 의사소통이 인코딩된 메시지의 통계 구조 사이는 차이가 있습니다. 이 차이를 검증하기 위해 다음과 같은 사고 실험을 해보죠. 컴퓨터가 대부분의 디지털 통신에서 하는 것처럼 사람의 언어가 의사소통을 압축하는데 더 뛰어나다면 어떨까요? 언어의 의미가 줄진 않지만 고유한 통계 구조가 사라질 것입니다. 이는 방금과 같은 언어 모델을 학습하는 것을 불가능하게 만듭니다.

- 정리

 * 이전의 토큰이 주어지면 다음 토큰(들)을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성할 수 있습니다.
 * 텍스트의 경우 이런 모델을 언어 모델이라 부릅니다. 단어 또는 글자 단위 모두 가능합니다.
 * 다음 토큰을 샘플링할 때 모델이 만든 출력에 집중하는 것과 무작위성을 주입하는 것 사이에 균형을 맞추어야 합니다.
 * 이를 위해 소프트맥스 온도 개념을 사용합니다. 항상 다양한 온도를 실험해서 적절한 값을 찾습니다.

In [28]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 61): # 60 에포크 동안 모델을 훈련
    print('에포크', epoch)
    model.fit(x, y, batch_size = 128, epochs = 1) # 데이터에서 한 번만 반복해서 모델을 학습

    seed_text = text[start_index: start_index + maxlen] # 무작위로 시드 텍스트를 선택
    print('--- 시드 텍스트:', seed_text)

    for temperature in [0.2, 0.5, 1.0, 1.2]: # 여러 가지 샘플링 온도를 시도
        print('--- 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        for i in range(400): # 시드 텍스트에서 시작해서 400개의 글자를 생성

            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꾼다.
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링
            preds = model.predict(sampled, verbose = 0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
    print()

에포크 1
1565/1565 [==============================] - 13s 7ms/step - loss: 1.9652
--- 시드 텍스트: the slowly ascending ranks and classes, in which,
through fo
--- 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the some as the self and the soul a sour and the self--and strongering the soul the self--is the sense of the self--present the sense of the some in the self-spirit and the sense of the expecient and the complesent and the comparied and strongering the self--and soul of the present and the present the sense of the self of the present of the some as the sense of the self--and the sense of the sou
--- 온도: 0.5
the slowly ascending ranks and classes, in which,
through for a think the see himself and pressed and one as the more one is the decises and yrease one the some the for out and in the preasing the duch the feelings, at the comparind that our the ender in all for an and the for of the greated and but compression of the selver the pertach or compularing the will 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


nly thinker. if thus they the principless or that the feeling of understan
--- 온도: 1.2
the slowly ascending ranks and classes, in which,
through forments of perceives, and was tleautses to
ourselver, corration, its power of a
kind of dementinatic sensations,
aware
he onwhether voluntary"--no sames for him acts and coursed to over love of the supervalmance, its philosophy is dif the
strange of slavance forse, as a little custom to foremooges forful on the drieu, renacures--en! speads and attempt of accomman bisgobary for
generated that lesson
에포크 33
1565/1565 [==============================] - 12s 7ms/step - loss: 1.2797
--- 시드 텍스트: the slowly ascending ranks and classes, in which,
through fo
--- 온도: 0.2
the slowly ascending ranks and classes, in which,
through formerly and all the consequently and the present of the belief in the consequently of the conscience of the state of the condition of the desirable of the state of the sense of the consequences of the condition of the same the p